<a href="https://colab.research.google.com/github/JonRMurphy/Home_Sales/blob/main/Starter_Code/Home_Sales_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.4.0'
spark_version = 'spark-3.5.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done


In [4]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [11]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"



In [33]:
# 2. Create a temporary view of the DataFrame.
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), sep=",", header=True, ignoreLeadingWhiteSpace=True)
df.createOrReplaceTempView('df')
df.show()

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [24]:
# 3. What is the average price for a four bedroom house sold per year, rounded to two decimal places?
from pyspark.sql.functions import mean, round

# Filter the DataFrame for four-bedroom houses
four_bedroom_df = df[df['bedrooms'] == 4]

# Group by 'Year' and calculate the average price for each year
average_price_per_year = four_bedroom_df.groupby('date').agg(round(mean('price'), 2).alias('Average Price'))

# Show the results
average_price_per_year.show()


+----------+-------------+
|      date|Average Price|
+----------+-------------+
|2020-04-13|     303153.3|
|2020-02-26|     295492.0|
|2019-08-22|     331777.3|
|2019-08-08|    305486.56|
|2021-11-03|    279481.33|
|2019-08-23|    273917.83|
|2020-06-24|    363142.22|
|2019-09-29|     286234.2|
|2021-12-23|     263829.0|
|2019-08-31|    367088.29|
|2019-04-25|    240981.75|
|2020-06-08|     269630.0|
|2020-11-12|    314268.22|
|2021-04-06|    250681.71|
|2022-05-17|    279781.33|
|2020-09-12|    265752.09|
|2022-03-30|     273248.0|
|2019-01-17|    333439.09|
|2021-08-30|     363022.0|
|2019-04-26|    318184.93|
+----------+-------------+
only showing top 20 rows



In [28]:
# 4. What is the average price of a home for each year the home was built,
# that have 3 bedrooms and 3 bathrooms, rounded to two decimal places?

# Filter the DataFrame for four-bedroom houses
three_bed_bath_df = df.filter((df['bedrooms'] == 3) & (df['bathrooms'] == 3))

# Group by 'Year' and calculate the average price for each year
average_price_three = three_bed_bath_df.groupby('date_built').agg(round(mean('price'), 2).alias('Average Price'))

# Show the results
average_price_three.show()


+----------+-------------+
|date_built|Average Price|
+----------+-------------+
|      2016|    290555.07|
|      2012|    293683.19|
|      2017|    292676.79|
|      2014|    290852.27|
|      2013|    295962.27|
|      2011|    291117.47|
|      2015|     288770.3|
|      2010|    292859.62|
+----------+-------------+



In [29]:
# 5. What is the average price of a home for each year the home was built,
# that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet, rounded to two decimal places?
house_df_4 = df.filter((df['bedrooms'] == 3) & (df['bathrooms'] == 3) & (df['floors'] == 2) & (df['sqft_living'] > 2000))

# Group by 'Year' and calculate the average price for each year
average_price_4 = house_df_4.groupby('date_built').agg(round(mean('price'), 2).alias('Average Price'))

# Show the results
average_price_4.show()


+----------+-------------+
|date_built|Average Price|
+----------+-------------+
|      2016|     293965.1|
|      2012|    307539.97|
|      2017|    280527.62|
|      2014|    297619.46|
|      2013|    303676.79|
|      2011|    276553.81|
|      2015|    297609.97|
|      2010|    285010.22|
+----------+-------------+



In [41]:
# 6. What is the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000? Order by descending view rating.
# Although this is a small dataset, determine the run time for this query.

# Group by 'Year' and calculate the average price for each year
home_sales = df.groupby('view').agg(round(mean('price'), 2).alias('Average Price'))

# Show the results
home_sales.show()

start_time = time.time()



print("--- %s seconds ---" % (time.time() - start_time))

+----+-------------+
|view|Average Price|
+----+-------------+
|   7|    288929.09|
|  51|    788128.21|
|  15|    284907.04|
|  54|    798684.82|
|  11|    280356.07|
|  29|    283881.72|
|  69|    750537.94|
|  42|    289225.45|
|  87|    1072285.2|
|  73|    752861.18|
|  64|    767036.67|
|   3|    284314.53|
|  30|    281085.62|
|  34|    286124.07|
|  59|     791453.0|
|   8|    279099.78|
|  22|    284908.42|
|  28|    285474.25|
|  85|   1056336.74|
|  16|    291990.83|
+----+-------------+
only showing top 20 rows

--- 0.00013017654418945312 seconds ---


In [43]:
# 7. Cache the the temporary table home_sales.
spark.sql("cache table df")

DataFrame[]

In [44]:
# 8. Check if the table is cached.
spark.catalog.isCached('df')

True

In [48]:
# 9. Using the cached data, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the uncached runtime.

start_time = time.time()

# Group by 'Year' and calculate the average price for each year
average_price_5 = df.groupby('view').agg(round(mean('price'), 2).alias('Average Price'))

# Show the results
average_price_5.show()

print("--- %s seconds ---" % (time.time() - start_time))


+----+-------------+
|view|Average Price|
+----+-------------+
|   7|    288929.09|
|  51|    788128.21|
|  15|    284907.04|
|  54|    798684.82|
|  11|    280356.07|
|  29|    283881.72|
|  69|    750537.94|
|  42|    289225.45|
|  87|    1072285.2|
|  73|    752861.18|
|  64|    767036.67|
|   3|    284314.53|
|  30|    281085.62|
|  34|    286124.07|
|  59|     791453.0|
|   8|    279099.78|
|  22|    284908.42|
|  28|    285474.25|
|  85|   1056336.74|
|  16|    291990.83|
+----+-------------+
only showing top 20 rows

--- 0.6975200176239014 seconds ---


In [45]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
df.write.partitionBy("date_built").mode("overwrite").parquet("df_partitioned")

In [46]:
# 11. Read the parquet formatted data.
p_df_p=spark.read.parquet('df_partitioned')

In [47]:
# 12. Create a temporary table for the parquet data.
p_df_p.createOrReplaceTempView('p_dfview_p')

In [49]:
# 13. Using the parquet DataFrame, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the cached runtime.

start_time = time.time()

# Group by 'Year' and calculate the average price for each year
average_price_5 = df.groupby('view').agg(round(mean('price'), 2).alias('Average Price'))

# Show the results
average_price_5.show()

print("--- %s seconds ---" % (time.time() - start_time))

+----+-------------+
|view|Average Price|
+----+-------------+
|   7|    288929.09|
|  51|    788128.21|
|  15|    284907.04|
|  54|    798684.82|
|  11|    280356.07|
|  29|    283881.72|
|  69|    750537.94|
|  42|    289225.45|
|  87|    1072285.2|
|  73|    752861.18|
|  64|    767036.67|
|   3|    284314.53|
|  30|    281085.62|
|  34|    286124.07|
|  59|     791453.0|
|   8|    279099.78|
|  22|    284908.42|
|  28|    285474.25|
|  85|   1056336.74|
|  16|    291990.83|
+----+-------------+
only showing top 20 rows

--- 0.4839637279510498 seconds ---


In [50]:
# 14. Uncache the home_sales temporary table.
spark.sql("uncache table df")

DataFrame[]

In [51]:
# 15. Check if the home_sales is no longer cached
spark.catalog.isCached("df")


False